# IBM Data Science Professional Certificate - Capstone Final Project - Oscar Antunes
##### 
### Analysis of Düsseldorf boroughs, Germany
##### 
#### Webscraping for  Boroughs, Houses available, Housing prices, Habitants, Venues
##### 


In [1]:
# Import required libraries for scrapping
import urllib.request
from bs4 import BeautifulSoup
import smtplib
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests


import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Regular Expressions to collect the addresses
import re

# Get the same number of District cells as Boroughs
from itertools import chain

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values



# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Class object has been created to store the credentials for safekeeping
#import credentials_oa_outlk

print('Libraries imported.')

Libraries imported.


#### 
## Getting the number of habitants per borough

In [2]:
url_hab = 'http://www.citypopulation.de/en/germany/dusseldorf/admin/'


In [3]:
page = requests.get(url_hab)
    
# Parsing through the URL
soup = BeautifulSoup(page.content, 'html.parser')


# The Category of borough can be obtained with:
# Borough Category -> <tbody class="admin2">
# Name of Borough -> <span itemprop="name">Altstadt</span>
# Pop 2018 -> <td class="rpop prio1">2,404</td>

#### 
## Getting the Coordinates for the city of Düsseldorf

In [4]:
# Getting the Coordinates for Düsseldorf

address = 'Düsseldorf, NRW'

geolocator = Nominatim(user_agent="duesseldorf_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Düsseldorf are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Düsseldorf are 51.2254018, 6.7763137.


# 
## Getting the Boroughs

In [5]:
url_wiki = 'https://de.wikipedia.org/wiki/Liste_der_Stadtbezirke_von_D%C3%BCsseldorf'

page = requests.get(url_wiki)
# Parsing through the URL
soup = BeautifulSoup(page.content, 'html.parser')


print("Data parsed.")

Data parsed.


In [6]:
# Identify the table to scrap
right_table = soup.find('table', class_ = 'wikitable sortable')



# 
## Assigning Districts and Boroughs to DataFrame

In [7]:
# SET THE DATAFRAME COLUMNS || CREATE EMPTY DATAFRAME WITH THE SAME STRUCTURE OF THE ORIGINAL
column_names = ['District','Borough','Fläche','Einwohner','Bewölkerungsdichte','Bezirkvorsteher','K','Karte']
df = pd.DataFrame(columns = column_names)

#####################################################
# LOAD OF THE DATAFRAME || CLEAR TO THE 2 COLUMNS NEEDEED
for tr_cell in right_table.find_all('tr'):
    row_data = []
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.rstrip())
        
    if len(row_data)>0:
        df.loc[len(df)] = row_data
        
df = df[['District', 'Borough']]
print(df.head(2),'\n')

#####################################################
# REMOVE NUMERIC DIGITS FROM THE BOROUGHS
df.set_index(['District', 'Borough'])  
df = df.replace(regex=r'[0-9]+', value=',')
district_numb = ['1','2','3','4','5','6','7','8','9','10']
df['district_numb'] = district_numb

print(df.head(5))

           District                                            Borough
0  Stadtbezirk 1[1]  011 Altstadt012 Carlstadt013 Stadtmitte014 Pem...
1  Stadtbezirk 2[2]     021 Flingern Süd022 Flingern Nord023 Düsseltal 

           District                                            Borough  \
0  Stadtbezirk ,[,]  , Altstadt, Carlstadt, Stadtmitte, Pempelfort,...   
1  Stadtbezirk ,[,]           , Flingern Süd, Flingern Nord, Düsseltal   
2  Stadtbezirk ,[,]  , Friedrichstadt, Unterbilk, Hafen, Hamm, Volm...   
3  Stadtbezirk ,[,]         , Oberkassel, Heerdt, Lörick, Niederkassel   
4  Stadtbezirk ,[,]  , Stockum, Lohausen, Kaiserswerth, Wittlaer, A...   

  district_numb  
0             1  
1             2  
2             3  
3             4  
4             5  


In [8]:
### Get several District cellsfor each Borough
# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = df['Borough'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
res = pd.DataFrame({'District': np.repeat(df['District'], lens),
                    'Borough': chainer(df['Borough']),
                    'district_numb': np.repeat(df['district_numb'], lens)})
# Strip empty cells ""
res = res[res['Borough'].str.strip().astype(bool)]

In [9]:
# Concatenate the District to include the Number for each District
res['Dis'] = res['District'].str[:11] 
res['District'] = res['Dis']+' '+res['district_numb']
df_boroughs = res[['District','Borough']]
df_boroughs.reset_index(drop=True)
print('Scrapping of Boroughs completed!\n')
print(df_boroughs.head(10))

Scrapping of Boroughs completed!

        District          Borough
0  Stadtbezirk 1         Altstadt
0  Stadtbezirk 1        Carlstadt
0  Stadtbezirk 1       Stadtmitte
0  Stadtbezirk 1       Pempelfort
0  Stadtbezirk 1        Derendorf
0  Stadtbezirk 1         Golzheim
1  Stadtbezirk 2     Flingern Süd
1  Stadtbezirk 2    Flingern Nord
1  Stadtbezirk 2        Düsseltal
2  Stadtbezirk 3   Friedrichstadt


# 
## Scrapping housing information

In [10]:
pnumber = 1 
num = 1 
address1 = []
price1 = []
rooms1 = []

while pnumber < 11:
    url_house = 'https://www.immobilienscout24.de/Suche/radius/wohnung-mieten?centerofsearchaddress=D%C3%BCsseldorf;;;1276010012;Nordrhein-Westfalen;&numberofrooms=2.0-&price=-2000.0&geocoordinates=51.23824;6.81513;3.0&pagenumber={}'.format(pnumber)
    headers = {"User-agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

    page = requests.get(url_house, headers=headers)
    
    # Parsing through the URL
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #Check if pages are being scraped
    print('Page {} scraped'.format(num))
    num = num + 1
    
    #Getting Adress
    address  = soup.find_all('button')
    
    for a in address:
        a = a.decode()
        stuff = re.findall(("[^<>]+\w+[A-Za-z].\\s+Düsseldorf"), a)
        if len(stuff) == 0:
            continue
        else:        
            address1.append(stuff)
    
    #Getting price
    price  = soup.find_all('dd')
    
    for p in price:
        p = p.decode()
        stuff = re.findall(("[^<>]+\\s+€"),p)
        if len(stuff) == 0:
            continue
        else:        
            price1.append(stuff)
    
    #Getting number of rooms
    rooms = soup.findAll(class_=['onlySmall'])
    
    for r in rooms:
        r = r.decode()
        stuff = re.findall(("[>]+[0-9]+[<]"), r)
        if len(stuff) == 0:
            continue
        else:   
            rooms1.append(stuff)
    
    
    pnumber = pnumber + 1
    

Page 1 scraped
Page 2 scraped
Page 3 scraped
Page 4 scraped
Page 5 scraped
Page 6 scraped
Page 7 scraped
Page 8 scraped
Page 9 scraped
Page 10 scraped


In [11]:
rooms1[:5]

[['>3<'], ['>2<'], ['>2<'], ['>2<'], ['>3<']]

In [12]:

flat_address = []
flat_price = []
flat_rooms = []

for sublist in address1:
    for item in sublist:
        flat_address.append(item)

for sublist in price1:
    for item in sublist:
        flat_price.append(item)
        
for sublist in rooms1:
    for item in sublist:
        flat_rooms.append(item)

pd_address = pd.Series(flat_address, dtype = object)
pd_price = pd.Series(flat_price, dtype = object)
pd_rooms = pd.Series(flat_rooms, dtype = object)


In [13]:
# Finishing cleaning the number of rooms
pd_rooms = pd_rooms.str[1]

In [14]:
result = pd.concat([pd_address, pd_price, pd_rooms], axis=1, sort=False)
result.columns = ['Address', 'Price', 'Num_Rooms']
result

,Address,Price,Num_Rooms
0,"Heideweg 5, Mörsenbroich, Düsseldorf","1.086,03 €",3
1,"Langerstraße 57, Flingern Süd, Düsseldorf",700 €,2
2,"Glockenstraße 35, Derendorf, Düsseldorf",600 €,2
3,"Mindener Straße 100, Oberbilk, Düsseldorf","1.403,27 €",2
4,"Hohenzollernallee 4, Flingern Nord, Düsseldorf","1.011,71 €",3
5,"Hohenzollernallee 2, Flingern Nord, Düsseldorf","1.291,46 €",4
6,"Hohenzollernallee 10, Flingern Nord, Düsseldorf","1.588,17 €",3
7,"Hohenzollernallee 12, Flingern Nord, Düsseldorf","1.306,42 €",2
8,"Hohenzollernallee 16, Flingern Nord, Düsseldorf","990,14 €",3
9,"Hohenzollernallee 16, Flingern Nord, Düsseldorf","1.435,17 €",2
